In [ ]:
# default_exp gh

# gh

> Tools for GitHub releases

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re
import yaml
import subprocess
import requests
import json
from fastcore.script import call_parse, Param, store_true
from easyrelease.utils import get_config, read_credentials, check_project_root, get_template

In [ ]:
#export
@check_project_root
def write_gh_template():
    "Write GitHub template for assigning commit messages to categories"
    template = get_template("gh-release-config-template.yaml")
    template = "\n".join(template.split())  # remove redundant whitespaces
    with open(".gh-release-config.yaml", "w") as f:
        f.write(template)

In [ ]:
#export
def get_last_tag():
    "Get latest git tag"
    return subprocess.run(["git", "describe", "--tags", "--abbrev=0"], 
                          capture_output=True).stdout.decode("utf").strip()

In [ ]:
#export
def get_commit_msgs(from_tag=None, to="HEAD"):
    "Get commits `from_tag` to `to`. If `from_tag` is None then it will get the commits from latest tag"
    last_tag = from_tag if from_tag is not None else get_last_tag()
    commit_msgs = subprocess.run(["git", "log", f"{last_tag}..{to}", "--pretty=%s"],
                          capture_output=True).stdout.decode("utf").strip().split("\n")
    return commit_msgs

In [ ]:
#export
def add_pull_request_link(msg):
    "Add pull request link to commit message `s` if reference is found, e.g. #100"
    cfg = get_config()
    git_url = cfg["git_url"]
    msg_with_link = re.sub("(#)(\d+)", rf"[\1\2]({git_url}/pull/\2)", msg)
    return msg_with_link

In [ ]:
#export
@check_project_root
def release_report(commit_msgs, report_title="", from_tag=None, to="HEAD"):
    "Get release report `from_tag` to commit `to` using `report_title` and commit messages `commit_msgs`"
    with open(".gh-release-config.yaml") as f:
        gh_config = yaml.safe_load(f)
    gh_categories = gh_config["gh_categories"]
    report_list = [report_title]
    for d in gh_categories:
        gh_cat_commits = [add_pull_request_link(f"* {msg}") for msg in commit_msgs if d["keyword"] in msg]
        gh_cat_report = f"{d['title']}\n" + "{}".format("\n".join(gh_cat_commits)) if len(gh_cat_commits) > 0 else ""
        report_list.append(gh_cat_report)
    report_list = [rep for rep in report_list if rep != ""]
    report = "\n\n".join(report_list)
    return report

In [ ]:
#export
@check_project_root
def get_gh_credentials():
    "Get GitHub credentials from file .gh-credentials"
    cfg = read_credentials(".gh-credentials")
    return (cfg["gh_user"], cfg["gh_token"])

In [ ]:
#export
def get_tags():
    "Get git tags as list"
    return (
        subprocess.run(["git", "tag", "-l"], capture_output=True)
        .stdout.decode("utf")
        .strip()
        .split("\n")
    )

In [ ]:
#export
@call_parse
def make_gh_release(draft: Param(help="Is this a release draft?", type=store_true)=False, 
                    prerelease: Param(help="Is this a prerelease?", type=store_true)=False):
    "Make a GitHub release. Indicate if there is a `draft` or a `prerelease` with default False both"
    cfg = get_config()  # package config
    gh_user, gh_token = get_gh_credentials()
    release_version = cfg["version"]
    repo_name = cfg["repo_name"]
    target_branch = cfg["branch"]
    commit_msgs = get_commit_msgs()
    report = release_report(commit_msgs=commit_msgs)
    # generate data for release
    gh_data = {
        "tag_name": release_version,
        "target_commitish": target_branch,
        "name": f"Release version {release_version}",
        "body": report,
        "draft": draft,
        "prerelease": prerelease
    }
    gh_api_url = f"https://api.github.com/repos/{repo_name}/releases"
    print("Creating GitHub release")
    r = requests.post(
        url=gh_api_url, data=json.dumps(gh_data), auth=HTTPBasicAuth(gh_user, gh_token)
    )
    print(f"Status code: {r.status_code}")
    print(r.text)
    # download new tag from GitHub
    subprocess.run(["git", "fetch"])

In [ ]:
#export
@check_project_root
def make_changelog():
    "Generate CHANGELOG.md file with release notes"
    tags = get_tags()
    changelog_title = "# Release notes"
    reports = []
    for from_tag, to in zip(tags[:-1], tags[1:]):
        commit_msgs = get_commit_msgs(from_tag=from_tag, to=to)
        reports.append(release_report(report_title=f"## {to}", commit_msgs=commit_msgs))
    changelog = changelog_title + "\n\n" + "\n\n".join(reports[::-1])
    with open("CHANGELOG.md", "w") as f:
        f.write(changelog)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_gh.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
